# Notebook demo of the possibilities and impossibilities of Kestrel in our case

## I. Add our data

For additional information on how work STIX and how connect your data on Kestrel, you can refer to the wiki of our github project

### Firstly, with stix-shifter

Initially, we created the stix-shifter configuration file for a stix-bundle and added the path to our json stix-bundle. We named the profile local_stix, but the call doesn’t work, and we are told that the configuration file cannot be found. However, we did add the path to the file in the required environment variable: KESTREL_STIXSHIFTER_CONFIG.
Thus, this method of adding it does not seem to work for stix-bundle.

In [1]:
data1 = GET network-traffic 
        FROM stixshifter://local-stix 
        WHERE src_port > 0

[ERROR] InvalidDataSource: invalid data source "local-stix" at interface "stixshifter". no local-stix configuration found
please check data source configuration.

### Secondly, with LOAD command

In Kestrel documentation, it is said that we can add file in our project by using the LOAD command, thus, we tried it with our stix files, but it isn't seen as STIX object and thus can't be processed by other commands as GET, the whole log file is seen as one object.

In [2]:
data2 = LOAD stix-objects/dataMessage.json

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),bundle*
data2,bundle,1,1,0


In [3]:
DISP data2

id,objects
bundle--4ab84de8-98c7-4a9c-9ff6-f39be7c79a8e,"{""type"":""observed-data"",""id"":""observed-data--a7db3b5c-6d1c-4b9e-8b3a-6f7e716c9fd1"",""created_by_ref"":""identity--b3a4c6e0-dc69-4f91-8154-31bfe1d85b37"",""created"":""2024-12-12T15:05:00.000Z"",""modified"":""2024-12-12T15:05:00.000Z"",""first_observed"":""2024-12-12T15:04:45.000Z"",""last_observed"":""2024-12-12T15:04:45.000Z"",""number_observed"":1,""objects"":{""0"":{""type"":""ipv4-addr"",""value"":""203.0.113.1"",""description"":""Sender's IP address""},""1"":{""type"":""ipv4-addr"",""value"":""192.168.100.2"",""description"":""Recipient's device IP address""},""2"":{""type"":""user-account"",""user_id"":""user12345"",""account_type"":""subscriber"",""display_name"":""John Doe"",""description"":""Subscriber receiving the SMS""},""3"":{""type"":""network-traffic"",""start"":""2024-12-12T15:04:45.000Z"",""end"":""2024-12-12T15:04:45.500Z"",""is_active"":false,""src_ref"":""0"",""dst_ref"":""1"",""protocols"":[""ip"",""udp""],""src_port"":5060,""dst_port"":2152,""description"":""Delivery of SMS over the 5G control plane""},""4"":{""type"":""artifact"",""mime_type"":""text\/plain"",""payload_bin"":""SGVsbG8sIGNvbW1lbnQgdGFsZSBhdXZyZSBsZSBuZXR3b3UgcXVpIGNob2lzaXJhIG5vdHJlIHNvdXJjZSA1Ryw="",""description"":""Contents of the received SMS message""}}}"


### The good method : with the GET command

Finally, it seems that the solution to process stix-bundles is with the GET command. In fact, we can add in the FROM field the path of our data and we can process it. However, we can't add all part of the STIX bundle, only a part of it. Thus, we must filter by a type, for instance network-traffic and therfore we can only access its attributes. Moreover, in the GET command, we need to add the WHERE field, if we want all datas, we can use a case that is allways matched as, for network-traffic entities, src_port > 0

In [4]:
data3 = GET network-traffic 
        FROM file:///home/vboxuser/Desktop/kestrel/stix-objects/logs_splunk_2.json 
        WHERE src_port > 0

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,bundle*,domain-name*,file*,ipv4-addr*,mac-addr*,network-traffic*,url*,user-account*
data3,network-traffic,1,1143,1143,0,1143,1143,2286,2286,1142,1143,1143


In [5]:
INFO data3

Entity Type,network-traffic
Number of Entities,1
Number of Records,1143
Entity Attributes,"dst_port, protocols, src_port, id"
Indirect Attributes,[]
Customized Attributes,
Birth Command,get
Associated Datasource,file:///home/vboxuser/Desktop/kestrel/stix-objects/logs_splunk_2.json
Dependent Variables,


#### Limitations :
1. It is impossible, for now, to access to the url/payload in the network entities for instance and all other entities created by a network traffic, only entities created by processus. However, we can access to the ip

In [6]:
urldata3 = FIND url CREATED BY data3

[ERROR] UnsupportedRelation: unsupported relation "network-traffic--created--url"
check for supported relations and entity types in the documentation.

In [7]:
urldata3 = FIND url ACCEPTED data3

[ERROR] UnsupportedRelation: unsupported relation "url--accepted--network-traffic"
check for supported relations and entity types in the documentation.

In [8]:
ipdata3 = FIND ipv4-addr ACCEPTED data3

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,bundle*,domain-name*,file*,ipv4-addr*,mac-addr*,network-traffic*,url*,user-account*
ipdata3,ipv4-addr,1,2286,2286,0,2286,2286,4571,4572,2286,2286,2286


 2. It is impossible to filter network entities using timestamps not indicated in the network-entities attributes (except timestamp with the START END field). Then, we can't, for instance, filter sms network-traffic by their ip sender (if we imagine that we have access to it).

In [9]:
ipofnetworktrafic = GET network-traffic 
                    FROM file:///home/vboxuser/Desktop/kestrel/stix-objects/dataMessage.json 
                    WHERE ipv4-addr != '0,0,0,0'

1:17: no viable alternative at input 'network-traffic:ipv4-addr'

In [10]:
test4 = FIND network-traffic FROM ipdata3 WHERE value != "0.0.0.0"

[ERROR] UnsupportedRelation: unsupported relation "network-traffic--from--ipv4-addr"
check for supported relations and entity types in the documentation.

Thus, in the context of our scenario, using only Kestrel, it is impossible, for example, to check all network traffic coming from the same person, one of the hypotheses we wanted to use to characterize a smishing attack.
Moreover, since we cannot access data beyond what is specified by Kestrel, we cannot, for instance, add phone numbers or SUPIs to identify a user.

## II. Process our data to see if our attack scenario is done 

#### The github repo kestrel-analytics give us some tools in order to process our data using python functions and docker container usable with the APPLY command

### Suspicious score done on our data

One of the function of kestrel analytics, which use open source databases of common attack to detect if a process or a network-entities is suspect and give it a score. This tool can be used in our case of detection of smishing. But for now, by seeing the source code, it seems that there are only rules related specifically to the process. However, it seems only possible to add our own rules locally by modifying the python file and the rules.json file but in that case, Kestrel will be useless and it will be more optimal to use directly python to access to all the data of the stix object and don't be limited. For now, for all of our attempts to be suspicious, we had a 0 suspicious score

In [11]:
data4 = GET network-traffic 
        FROM file:///home/vboxuser/Desktop/kestrel/stix-objects/dataMessages.json 
        WHERE src_port > 0

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,bundle*,domain-name*,file*,ipv4-addr*,mac-addr*,network-traffic*,url*,user-account*
data4,network-traffic,3,4,4,0,0,0,7,0,1,0,4


In [12]:
APPLY docker://susp_scoring ON data4
DISP data4

dst_port,protocols,src_port,id,start,end,is_active,description,x_suspicious_score,dst_ref.value,dst_ref.id,dst_ref.description,src_ref.value,src_ref.id,src_ref.description
2152,"[""ip"",""udp""]",5060,network-traffic--4e833943-b0b4-5404-a95c-887af3da80af,2024-12-12T15:04:45.000Z,2024-12-12T15:04:45.500Z,0,Delivery of SMS over the 5G control plane,0,None,None,None,None,None,None
2152,"[""ip"",""udp""]",5060,network-traffic--5529dd7a-9c7f-599d-b89f-0c090e761caf,2024-12-12T15:04:45.000Z,2024-12-12T15:04:45.500Z,0,Delivery of SMS over the 5G control plane,0,192.168.100.4,ipv4-addr--4adf17c5-1b00-5546-a6bd-e85e8c7a5e03,Recipient's device IP address,None,None,None
2152,"[""ip"",""udp""]",5060,network-traffic--e7bee814-1b5c-53e4-8e85-58fff53483cd,2024-12-12T15:04:45.000Z,2024-12-12T15:04:45.500Z,0,Delivery of SMS over the 5G control plane,0,203.0.113.1,ipv4-addr--ad74bc22-43d2-52ed-890e-7d382f58e1eb,Sender's IP address,None,None,None


### Domain Enrichment 

This function can be useful in our case to identify the domain name of each ip in order to be more readable by a human than monitor the data

In [13]:
APPLY python://domain-enrichment ON data4
DISP data4

dst_port,protocols,src_port,id,start,end,is_active,description,x_suspicious_score,x_domain_name,x_domain_organization,dst_ref.value,dst_ref.id,dst_ref.description,src_ref.value,src_ref.id,src_ref.description
2152,"[""ip"",""udp""]",5060,network-traffic--4e833943-b0b4-5404-a95c-887af3da80af,2024-12-12T15:04:45.000Z,2024-12-12T15:04:45.500Z,0,Delivery of SMS over the 5G control plane,0,,,None,None,None,None,None,None
2152,"[""ip"",""udp""]",5060,network-traffic--5529dd7a-9c7f-599d-b89f-0c090e761caf,2024-12-12T15:04:45.000Z,2024-12-12T15:04:45.500Z,0,Delivery of SMS over the 5G control plane,0,,,192.168.100.4,ipv4-addr--4adf17c5-1b00-5546-a6bd-e85e8c7a5e03,Recipient's device IP address,None,None,None
2152,"[""ip"",""udp""]",5060,network-traffic--e7bee814-1b5c-53e4-8e85-58fff53483cd,2024-12-12T15:04:45.000Z,2024-12-12T15:04:45.500Z,0,Delivery of SMS over the 5G control plane,0,,,203.0.113.1,ipv4-addr--ad74bc22-43d2-52ed-890e-7d382f58e1eb,Sender's IP address,None,None,None


### Attribute plot

This function is used to plot attributes. In our case, we can use it to plot and  see visually if their are an anormal number of messages send at a moment. Or plot all the occurences of an ip .

Plot of the sms during the time

In [14]:
data5 = GET network-traffic
            FROM file:///home/vboxuser/Desktop/kestrel/stix-objects/lab101.json
            WHERE dst_port > 0
            START 2021-04-03T00:00:00Z STOP 2021-04-04T00:00:00Z


data5_ts = TIMESTAMPED(data5)
        
APPLY python://attribute-plot ON data5_ts WITH XPARAM=first_observed, YPARAM=id

dst_ref.description:1

Occurences by ip : 

In [15]:
APPLY docker://attrib-plot ON data4 with XPARAM=dst_ref

[ERROR] AnalyticsError: attrib-plot failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'dst_ref'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/analytics/analytics.py", line 222, in <module>
    fig = analytics(dfi)
  File "/opt/analytics/analytics.py", line 203, in analytics
    x_ftype = feature_type(df, x_col)
  File "/opt/analytics/analytics.py", line 53, in feature_type
    if str(df[column].dtype).star

### GROUP COMMAND

As in SQL, we can group data with some similar particularities

In [16]:
testv5g = GROUP data4 BY src_port

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,bundle*,directory*,domain-name*,file*,ipv4-addr*,ipv6-addr*,mac-addr*,network-traffic*,process*,url*,user-account*,x-ecs-destination*,x-ecs-network*,x-ecs-process*,x-ecs-source*,x-ecs-user*,x-oca-asset*,x-oca-event*
testv5g,network-traffic,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## III. What is now impossible with Kestrel

Currently, the use of Kestrel seems unfit with 5g data and thus with our 5g attack scenario. In fact, as seen before, we are very limited with the stix entities now defined and the attributes linked to them which limit us to correctly detect an attack.
For instance, we can't use the payload of our logs and therefore it make its nearly impossible to establish some stages of the attack scenario. 
In its current state, Kestrel does not appear to be the optimal choice for this project. Python, for example, seems more suitable for threat hunting on 5G networks. Therefore, we do not recommend using Kestrel for this purpose.